# Measuring impact of tolerances on Gurobi

Following the first analysis, I slightly improved the numerics of the model through scaling and I am assessing the performance again in here. The numerical ranges now look like this:

```
Matrix range     [1e-05, 4e+02]
Objective range  [1e+00, 1e+00]
Bounds range     [5e-02, 2e+03]
RHS range        [7e-03, 2e+03]
```

With a matrix range of 1e7 and an RHS range of 1e6 they are far from being perfect, but better than last time.

In here we analyse the result of Gurobi runs using the same model but different parameter options. I used the `baseline` scenario and changed the `OptimalityTol` and `FeasibilityTol` values of Gurobi to:

* 1e-6 (default value)
* 1e-5
* 1e-4
* 1e-3
* 1e-2 (max value)

The hope being, that the result remains the same (remains close enough) but that solution time drops. Let's see whether that actually worked.

In [1]:
from pathlib import Path
import re

from scipy import stats

In [2]:
SOLVE_DURATION_PREFIX = "SOLVER   Solved in"
SOLUTION_PREFIX = "SOLVER   Optimal objective"


def _preprocess_log(path_to_log):
    path_to_log = Path(path_to_log)
    with path_to_log.open("r") as log_file:
        lines = log_file.readlines()
    return [line[22:] for line in lines]
    

def parse_solution_time(path_to_log):
    lines = _preprocess_log(path_to_log)
    duration_line = list(filter(lambda line: line.startswith(SOLVE_DURATION_PREFIX), lines))[0]
    return float(re.findall('\d+.?\d+', duration_line)[1])


def parse_iterations(path_to_log):
    lines = _preprocess_log(path_to_log)
    iterations_line = list(filter(lambda line: line.startswith(SOLVE_DURATION_PREFIX), lines))[0]
    return float(re.findall('\d+.?\d+', iterations_line)[0])


def parse_solution(path_to_log):
    lines = _preprocess_log(path_to_log)
    solution_line = list(filter(lambda line: line.startswith(SOLUTION_PREFIX), lines))[0]
    return float(re.findall('\d+.?\d+e?\+?\d+', solution_line)[0])

In [3]:
def analyse_runs(path_to_root_folder):
    durations = [parse_solution_time(path) for path in Path(path_to_root_folder).glob("*.log")]
    solutions = [parse_solution(path) for path in Path(path_to_root_folder).glob("*.log")]
    iterations = [parse_iterations(path) for path in Path(path_to_root_folder).glob("*.log")]
    
    mean, std = stats.norm.fit(solutions)
    print(f"Gurobi found the following optimal value on average: {mean} (std {std}).")
    mean, std = stats.norm.fit(iterations)
    print(f"Gurobi needed on average {mean:.0f} (std {std:.0f}) iterations to find the optimal value.")
    mean, std = stats.norm.fit(durations)
    print(f"Gurobi needed on average {mean:.0f}s (std {std:.0f}s) to find the optimal value.")

    
def diff(path_to_root_folder, path_to_root_folder_of_base):
    durations = [parse_solution_time(path) for path in Path(path_to_root_folder).glob("*.log")]
    solutions = [parse_solution(path) for path in Path(path_to_root_folder).glob("*.log")]
    iterations = [parse_iterations(path) for path in Path(path_to_root_folder).glob("*.log")]
    
    base_durations = [parse_solution_time(path) for path in Path(path_to_root_folder_of_base).glob("*.log")]
    base_solutions = [parse_solution(path) for path in Path(path_to_root_folder_of_base).glob("*.log")]
    base_iterations = [parse_iterations(path) for path in Path(path_to_root_folder_of_base).glob("*.log")]
    
    mean, std = stats.norm.fit(solutions)
    mean_base, std_base = stats.norm.fit(base_solutions)
    print(f"Relative diff of objective average to base: {mean / mean_base}")
    mean, std = stats.norm.fit(iterations)
    mean_base, std_base = stats.norm.fit(base_iterations)
    print(f"Relative diff of iterations average to base: {mean / mean_base}")
    mean, std = stats.norm.fit(durations)
    mean_base, std_base = stats.norm.fit(base_durations)
    print(f"Relative diff of durations average to base: {mean / mean_base}")
    

In [4]:
print("Tolerances e-6")
print("--------------")
analyse_runs("minus6/")
print()
print("Tolerances e-5")
print("--------------")
analyse_runs("minus5/")
diff("minus5/", "minus6/")
print()
print("Tolerances e-4")
print("--------------")
analyse_runs("minus4/")
diff("minus4/", "minus6/")
print()
print("Tolerances e-3")
print("--------------")
print("All runs needed more than 24h and were killed by the cluster.")
print()
print("Tolerances e-2")
print("--------------")
print("All runs needed more than 24h and were killed by the cluster.")
print()

Tolerances e-6
--------------
Gurobi found the following optimal value on average: 336375.605 (std 0.0).
Gurobi needed on average 2170133 (std 0) iterations to find the optimal value.
Gurobi needed on average 11054s (std 1709s) to find the optimal value.

Tolerances e-5
--------------
Gurobi found the following optimal value on average: 336375.602 (std 0.0).
Gurobi needed on average 837156 (std 0) iterations to find the optimal value.
Gurobi needed on average 1223s (std 200s) to find the optimal value.
Relative diff of objective average to base: 0.9999999910813985
Relative diff of iterations average to base: 0.38576253160520574
Relative diff of durations average to base: 0.11060621371797613

Tolerances e-4
--------------
Gurobi found the following optimal value on average: 336375.62600000005 (std 5.820766091346741e-11).
Gurobi needed on average 616935 (std 0) iterations to find the optimal value.
Gurobi needed on average 13482s (std 1972s) to find the optimal value.
Relative diff of ob

## Results

Whenever a result was found, the solution was close to the reference. So it seems save to play around with the two parameters. In terms of runtime, there seems to be a sweet spot at 1e-5: compared to the reference, the result was found 10 times faster. Compared to the worst performance, it was even almost 100 times faster. It is not intuitive that the performance becomes better moving from 1e-6 to 1e-5 and then becomes worse and worse the larger the tolerances gets. Possible explanations may be (a) that it's the combination of the two parameters, (b) that it's due to the remaining numerical difficulties of the problem, or (c) that it's unrelated to the parameters values. Further tests would be necessary to verify or deny these hypotheses.

## Conclusion

It's difficult to give advices based on these results. What I can say is that the performance is drastically impacted by the two parameters, and the correcteness of the solution does seem to be very insensitive to the parameters.

It's impossible to suggest a certain parameter value based on the results. But, because the variation between the results for same parameter values isn't huge, it may make sense to test the parameter values for a certain model. Either on a reduced model before running the full model, or on the full model before running sensitivitiy analyses with loads of runs.